# Facts up Front Testing Notebook

In [1]:
# Python program to illustrate
# template matching
import cv2
import numpy as np
import imutils
from imutils.object_detection import non_max_suppression
from pathlib import Path
import glob
import pandas as pd

In [2]:
def object_finder(img,temp,viz):

    found = None
# ------Multiscaling-----------------
    # loop over the scales of the image
    edge=cv2.Canny(img,50,200)
    for scale in np.linspace(0.1, 1.0, 20)[::-1]:
        # resize the image according to the scale, and keep track
        # of the ratio of the resizing
        resized = cv2.resize(temp, dsize = (0,0), fx = scale, fy = scale)
        r=1
#         r = temp.shape[1] / float(resized.shape[1])
        # if the resized image is smaller than the template, then break
        # from the loop
        if resized.shape[0] < tH or resized.shape[1] < tW:
            break
        temp=cv2.Canny(resized,50,200)
        result = cv2.matchTemplate(edge,temp, cv2.TM_CCORR_NORMED)
        (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)
        # check to see if the iteration should be visualized
    #     if args.get("visualize", False):
    #         # draw a bounding box around the detected region
    #         clone = np.dstack([edged, edged, edged])
    #         cv2.rectangle(clone, (maxLoc[0], maxLoc[1]),
    #             (maxLoc[0] + tW, maxLoc[1] + tH), (0, 0, 255), 2)
    #         cv2.imshow("Visualize", clone)
    #         cv2.waitKey(0)
    #     # if we have found a new maximum correlation value, then update
    #     # the bookkeeping variable
        if found is None or maxVal > found[0]:
            found = (maxVal, maxLoc, r)
        
    if viz==1:    
            # unpack the bookkeeping variable and compute the (x, y) coordinates
        # of the bounding box based on the resized ratio
        (_, maxLoc, r) = found
        (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
        (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
        # draw a bounding box around the detected result and display the image
        cv2.rectangle(img, (startX, startY), (endX, endY), (4, 255, 46), 2)
        cv2.namedWindow("Rescaled",cv2.WINDOW_NORMAL)
        cv2.imshow("Rescaled", img)
        k = cv2.waitKey(0)
        if k == 27:         # wait for ESC key to exit
            cv2.destroyAllWindows()
        # cv2.imshow("Image", img)
        # cv2.waitKey(0)
                    
    return found

In [28]:
img_folder=Path('C:/Users/G670813/OneDrive - General Mills/ITQ-prasad/itq-analytics/Computer Vision/Facts_Up_Front/test_images')
ref_folder=(r'C:\Users\G670813\OneDrive - General Mills\ITQ-prasad\itq-analytics\Computer Vision\Facts_Up_Front\reference image\ref4.jpg')

In [29]:
ref_image=cv2.imread(ref_folder,0)
ref_image=cv2.resize(ref_image, (0, 0), fx = 1, fy = 1)
# temp=cv2.Canny(ref_image,50,200)
(tH, tW) = ref_image.shape[:2]
cv2.imshow("template", ref_image)
cv2.waitKey(0)
cv2.destroyWindow('template')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'template' in function 'cvDestroyWindow'


In [30]:
images = Path(img_folder).glob('*.jpg')
detected_df=pd.DataFrame(columns=['img_name','result','flag'])
for image in images:
    img=cv2.imread(str(image.absolute()),0)
    img=cv2.resize(img, (0, 0), fx = 1, fy = 1)
    result = object_finder(img,ref_image,1)
    break
    if result[0]>0.5:
        flag="Y"
    else: 
        flag="N"
    detected_df=detected_df.append({'img_name':str(image),'result':str(result),'flag':str(flag)}, ignore_index=True)
    

In [20]:
detected_df

,img_name,result,flag
0,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.14037083089351654, (410, 466), 1)",N
1,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.19682733714580536, (101, 32), 1)",N
2,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.2889622747898102, (155, 214), 1)",N
3,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.28732505440711975, (155, 216), 1)",N
4,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.21144971251487732, (1817, 817), 1)",N
5,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.2931809425354004, (1255, 85), 1)",N
6,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.2865169048309326, (127, 42), 1)",N
7,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.288140207529068, (208, 46), 1)",N
8,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.28696203231811523, (346, 976), 1)",N
9,C:\Users\G670813\OneDrive - General Mills\ITQ-...,"(0.28707361221313477, (1851, 39), 1)",N


In [10]:
cv2.namedWindow('rescaled',cv2.WINDOW_NORMAL)
cv2.imshow("rescaled", result)
cv2.waitKey(0)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [54]:
type(images)

generator

In [12]:
detected_df.to_excel("detection.xlsx")

In [88]:
cv2.imshow("img", img)
cv2.waitKey(0)

-1